# Importing Libraries and Dataset

In [2]:
#Import Libraries
import pandas as pd
import numpy as np

from sklearn import metrics as skmetrics

import keras
import tensorflow
import tensorflow as tf
from keras import metrics
from keras.models import Sequential
from keras.layers import Dense, Input, Embedding, SimpleRNN, Dropout, LSTM, Bidirectional, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam, SGD
from keras import layers

In [4]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [5]:
X_smote = pd.read_csv('data/X_smote.csv')
y_smote = pd.read_csv('data/y_smote.csv')
X_val = pd.read_csv('data/X_val.csv')
y_val = pd.read_csv('data/y_val.csv')
X_test = pd.read_csv('data/X_test.csv')
y_test = pd.read_csv('data/y_test.csv')

# Evaluation Metrics Function

In [6]:
def print_statistics(y_actual, y_pred, y_prob, dataset_type):
  print(f"====================={dataset_type}====================")
  print(f"accuracy: {round(skmetrics.accuracy_score(y_actual, y_pred),5)}")
  print(f"precision: {round(skmetrics.precision_score(y_actual, y_pred),5)}")
  print(f"recall: {round(skmetrics.recall_score(y_actual, y_pred),5)}")
  print(f"f1 score: {round(skmetrics.f1_score(y_actual, y_pred),5)}")
  prec, recall, thresholds = skmetrics.precision_recall_curve(y_actual, y_prob)
  print(f"pr auc score: {round(skmetrics.auc(recall, prec),5)}")

# Neural Network Models

## Basic NN

In [15]:
# MODEL FOR 5 LAYERS
def train_base_model(n, dropout, X_train, y_train, X_val, y_val):
    model = Sequential()
    model.add(Dense(units=n, activation='relu', input_dim=176))
    
    model.add(Dropout(dropout)) 
    model.add(Dense(n, activation='relu')) # 1 hidden layer 
    model.add(Dropout(dropout)) 
    model.add(Dense(n/2, activation='relu')) # 2 hidden layer 
    model.add(Dropout(dropout)) 
    model.add(Dense(n/4, activation='relu')) # 3 hidden layer 
    model.add(Dropout(dropout)) 
    # model.add(Dense(n/6, activation='relu')) # 4 hidden layer 
    # model.add(Dropout(dropout)) 
    # model.add(Dense(n/8, activation='relu')) # 5 hidden layer 
    # model.add(Dropout(dropout)) 
    # model.add(Dense(n/16, activation='relu')) # 6 hidden layer 
    model.add(Dropout(dropout)) 
    model.add(Dense(units=1, activation='sigmoid')) #output layer sigmoid as binary
    model.summary()
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=["accuracy", metrics.Precision(),metrics.Recall()])
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    model.fit(X_train, y_train, batch_size=128, epochs=10, callbacks=[callback], validation_data=(X_val, y_val))
    return model

In [16]:
model_train = train_base_model(n=4080, dropout=0.1, X_train=X_smote, y_train=y_smote, X_val= X_val, y_val=y_val)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 4080)              722160    
                                                                 
 dropout_7 (Dropout)         (None, 4080)              0         
                                                                 
 dense_9 (Dense)             (None, 4080)              16650480  
                                                                 
 dropout_8 (Dropout)         (None, 4080)              0         
                                                                 
 dense_10 (Dense)            (None, 2040)              8325240   
                                                                 
 dropout_9 (Dropout)         (None, 2040)              0         
                                                                 
 dense_11 (Dense)            (None, 1020)             

In [17]:
# Evaluating Train and Test
train_pred_proba = model_train.predict(X_smote)
train_pred = np.round(train_pred_proba)
val_pred_proba = model_train.predict(X_val)
val_pred = np.round(val_pred_proba)
test_pred_proba = model_train.predict(X_test)
test_pred = np.round(test_pred_proba)

91/91 [==============================] - 0s 2ms/step


In [14]:
print_statistics(y_smote, train_pred, train_pred_proba, 'train')
print_statistics(y_val, val_pred, val_pred_proba, 'val')
print_statistics(y_test, test_pred, test_pred_proba, 'test')

=====================train====================
accuracy: 0.54786
precision: 0.52643
recall: 0.95339
f1 score: 0.67831
pr auc score: 0.47793
=====================val====================
accuracy: 0.36272
precision: 0.2914
recall: 0.94987
f1 score: 0.44599
pr auc score: 0.15276
=====================test====================
accuracy: 0.36827
precision: 0.29504
recall: 0.96401
f1 score: 0.45181
pr auc score: 0.15283
